# Synthetic Data Vault (SDV)
In this notebook, we will use Synthetic Data Vault (SDV) to create sequential synthetic data. The foundation of this notebook is the [tutorial from SDV](https://colab.research.google.com/drive/1cT4-jFK2Bxe93QudC_CwHq_yVCcNcxal?usp=sharing) and the notebook on DataSynthesizer.

In [1]:
# import relevant modules
import sdv
import pandas as pd
import numpy as np
import os

# Load data

In [2]:
data = pd.read_csv('CMAPSS/train_FD001.txt', sep=" ", header=None)

# drop last two columns with N/A values
data = data.iloc[:, :-2]

# rename columns according to readme.txt
col_names = ["unit_nr", "timecycle", "ops_set1", "ops_set2", "ops_set3"]
for i in range(1,22):
    col_names.append(f"sens_{i}")
data.columns = col_names

# Compute Remaining Useful Life (RUL) for each index (engine)
def add_remaining_useful_life(df):
    # Get the total number of cycles for each unit
    grouped_by_unit = df.groupby(by="unit_nr")
    max_cycle = grouped_by_unit["timecycle"].max()
    
    # Merge the max cycle back into the original frame
    result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='unit_nr', right_index=True)
    
    # Calculate remaining useful life for each row
    remaining_useful_life = result_frame["max_cycle"] - result_frame["timecycle"]
    result_frame["RUL"] = remaining_useful_life
    
    # drop max_cycle as it's no longer needed
    result_frame = result_frame.drop("max_cycle", axis=1)
    return result_frame

data = add_remaining_useful_life(data)

data.to_csv('CMAPSS/train_FD001_pre.csv', index=False)

data_length = len(data)

# display data
data
data.columns

Index(['unit_nr', 'timecycle', 'ops_set1', 'ops_set2', 'ops_set3', 'sens_1',
       'sens_2', 'sens_3', 'sens_4', 'sens_5', 'sens_6', 'sens_7', 'sens_8',
       'sens_9', 'sens_10', 'sens_11', 'sens_12', 'sens_13', 'sens_14',
       'sens_15', 'sens_16', 'sens_17', 'sens_18', 'sens_19', 'sens_20',
       'sens_21', 'RUL'],
      dtype='object')

# Create synthetic data

In [3]:
# Create metadata
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()

# load data
metadata.detect_from_dataframe(data=data)

# set unit-nr as sequence key, timecycle as sequence index
metadata.update_column(
    column_name='unit_nr',
    sdtype='id')
metadata.set_sequence_key('unit_nr')
metadata.set_sequence_index('timecycle')

# show metadata
# metadata

In [4]:
"""
In this example, we do not need context_columns.
As all columns have an alternating value, none
remains constant or context-dependent.

"""
from sdv.sequential import PARSynthesizer

# create synthesizer
synthesizer = PARSynthesizer(
    metadata,
    epochs=100,
    verbose=True
    )

synthesizer.fit(data)

Epoch 100 | Loss -0.4249756932258606: 100%|███| 100/100 [02:08<00:00,  1.29s/it]


In [5]:
# find amount of unique unit-nrs
n_unitnrs = len(data['unit_nr'].unique())

# create synthetic data with that same amount of unit-nrs
synthetic_data = synthesizer.sample(num_sequences=n_unitnrs)
synthetic_data

# export SyntData to folder
synthetic_data.to_csv('./CMAPSS/Synthetic/SDV_FD001_RUL.csv')

100%|█████████████████████████████████████████| 100/100 [01:11<00:00,  1.41it/s]


# Analysis
Cool! We've created our first synthetic data, now we wish to analyse this synthetic data. More specifically, we would like to compare the output and distributions of our synthetic data compared to the original data.

In [6]:
data_description = data.describe()
data_description

,unit_nr,timecycle,ops_set1,ops_set2,ops_set3,sens_1,sens_2,sens_3,sens_4,sens_5,...,sens_13,sens_14,sens_15,sens_16,sens_17,sens_18,sens_19,sens_20,sens_21,RUL
count,20631.000000,20631.000000,20631.000000,20631.000000,20631.0,20631.00,20631.000000,20631.000000,20631.000000,2.063100e+04,...,20631.000000,20631.000000,20631.000000,2.063100e+04,20631.000000,20631.0,20631.0,20631.000000,20631.000000,20631.000000
mean,51.506568,108.807862,-0.000009,0.000002,100.0,518.67,642.680934,1590.523119,1408.933782,1.462000e+01,...,2388.096152,8143.752722,8.442146,3.000000e-02,393.210654,2388.0,100.0,38.816271,23.289705,107.807862
std,29.227633,68.880990,0.002187,0.000293,0.0,0.00,0.500053,6.131150,9.000605,1.776400e-15,...,0.071919,19.076176,0.037505,1.387812e-17,1.548763,0.0,0.0,0.180746,0.108251,68.880990
min,1.000000,1.000000,-0.008700,-0.000600,100.0,518.67,641.210000,1571.040000,1382.250000,1.462000e+01,...,2387.880000,8099.940000,8.324900,3.000000e-02,388.000000,2388.0,100.0,38.140000,22.894200,0.000000
25%,26.000000,52.000000,-0.001500,-0.000200,100.0,518.67,642.325000,1586.260000,1402.360000,1.462000e+01,...,2388.040000,8133.245000,8.414900,3.000000e-02,392.000000,2388.0,100.0,38.700000,23.221800,51.000000
50%,52.000000,104.000000,0.000000,0.000000,100.0,518.67,642.640000,1590.100000,1408.040000,1.462000e+01,...,2388.090000,8140.540000,8.438900,3.000000e-02,393.000000,2388.0,100.0,38.830000,23.297900,103.000000
75%,77.000000,156.000000,0.001500,0.000300,100.0,518.67,643.000000,1594.380000,1414.555000,1.462000e+01,...,2388.140000,8148.310000,8.465600,3.000000e-02,394.000000,2388.0,100.0,38.950000,23.366800,155.000000
max,100.000000,362.000000,0.008700,0.000600,100.0,518.67,644.530000,1616.910000,1441.490000,1.462000e+01,...,2388.560000,8293.720000,8.584800,3.000000e-02,400.000000,2388.0,100.0,39.430000,23.618400,361.000000


In [7]:
syn_description = synthetic_data.describe()
syn_description

,unit_nr,timecycle,ops_set1,ops_set2,ops_set3,sens_1,sens_2,sens_3,sens_4,sens_5,...,sens_13,sens_14,sens_15,sens_16,sens_17,sens_18,sens_19,sens_20,sens_21,RUL
count,18564.000000,18564.000000,18564.000000,18564.000000,18564.0,18564.00,18564.000000,18564.000000,18564.000000,1.856400e+04,...,18564.000000,18564.000000,18564.000000,1.856400e+04,18564.000000,18564.0,18564.0,18564.000000,18564.000000,18564.000000
mean,52300.959491,101.884292,-0.000005,0.000004,100.0,518.67,642.752021,1591.245154,1410.282246,1.462000e+01,...,2388.101472,8146.460691,8.446927,3.000000e-02,393.414727,2388.0,100.0,38.796656,23.273122,98.710946
std,30224.243654,69.723950,0.001011,0.000149,0.0,0.00,0.285639,3.321873,5.296097,3.552809e-15,...,0.035969,11.956419,0.020248,1.040862e-17,0.917487,0.0,0.0,0.101342,0.059943,33.933783
min,297.000000,1.000000,-0.003940,-0.000590,100.0,518.67,641.606238,1577.999235,1390.250104,1.462000e+01,...,2387.948887,8099.940000,8.363616,3.000000e-02,390.000000,2388.0,100.0,38.366522,23.052963,0.000000
25%,20604.000000,47.000000,-0.000668,-0.000093,100.0,518.67,642.563708,1589.167107,1406.853930,1.462000e+01,...,2388.080701,8138.860253,8.435490,3.000000e-02,393.000000,2388.0,100.0,38.730902,23.233326,76.000000
50%,57382.000000,93.000000,-0.000009,0.000002,100.0,518.67,642.741310,1590.986394,1410.020844,1.462000e+01,...,2388.096152,8145.760799,8.443472,3.000000e-02,393.000000,2388.0,100.0,38.804221,23.276960,100.000000
75%,75683.000000,141.000000,0.000660,0.000100,100.0,518.67,642.940499,1593.400952,1413.857345,1.462000e+01,...,2388.123623,8154.348230,8.459784,3.000000e-02,394.000000,2388.0,100.0,38.859594,23.311051,120.000000
max,99455.000000,362.000000,0.004212,0.000584,100.0,518.67,643.919006,1606.104940,1431.071379,1.462000e+01,...,2388.285619,8209.460559,8.541943,3.000000e-02,397.000000,2388.0,100.0,39.212197,23.503433,234.000000


What directly stands out is that the unit-nrs are not ordered from 1-100. However, this is due to the very nature of SDV: it will never create instances that overlap with the original data [source](https://colab.research.google.com/drive/1YLk2uwn8yrSRPy0soEeJwu8Hdk_tGTlE#scrollTo=IjvfTOMVKQot). Therefore, this behaviour is to be expected.

The descriptions look rather similar, most means and standarddeviations are close to the description of the original data. Therefore it would be interesting to test the SDV data on a ML task.

## SDV evaluation

In [8]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    data,
    synthetic_data,
    metadata
)

# with 128 epochs:
"""Overall Quality Score: 81.07%

Properties:
- Column Shapes: 82.68%
- Column Pair Trends: 79.45%
"""

Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|█| 351/351 [00:00<00:00, 510.20it/s]

Overall Quality Score: 80.91%

Properties:
- Column Shapes: 82.84%
- Column Pair Trends: 78.98%


'Overall Quality Score: 81.07%\n\nProperties:\n- Column Shapes: 82.68%\n- Column Pair Trends: 79.45%\n'

# Testing properties